In [29]:
import pandas as pd

In [30]:
import numpy as np

In [31]:
df = pd.read_excel("sp500.xlsx")

In [32]:
df['Date'] = df['Date'].astype('datetime64')

In [33]:
df.set_index("Date", inplace=True)

In [34]:
df.sort_index(inplace=True)

In [ ]:
df.fillna(method="ffill", inplace=True)

In [36]:
dailyret = df.pct_change()

In [37]:
dailyret.iloc[-5:, :]

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,AAP,AMD,...,WHR,WMB,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-01,-0.003483,0.027090,0.005793,0.017139,-0.003104,-0.003603,-0.003586,-0.009032,-0.000478,0.003581,...,0.001433,0.003305,0.011551,0.003719,0.006356,0.011737,-0.002014,0.002817,-0.037330,0.021504
2023-05-02,-0.027114,-0.005133,0.003510,-0.013597,-0.009591,-0.018468,-0.029306,-0.014673,-0.010760,0.002453,...,-0.005651,-0.034914,-0.020761,-0.015249,0.000191,0.004711,-0.111359,0.030109,-0.108128,-0.014814
2023-05-03,-0.001457,-0.004586,0.002332,-0.015631,-0.005637,-0.010132,-0.006621,-0.063500,-0.002578,-0.092203,...,0.015468,-0.011604,-0.034187,-0.008683,-0.003540,-0.039118,0.011979,-0.023705,-0.052676,0.004692
2023-05-04,-0.009628,-0.003887,-0.008053,-0.012663,-0.031543,-0.008507,-0.010664,-0.027285,-0.028756,0.061137,...,-0.038257,0.004834,-0.000732,0.008905,0.020931,-0.007210,0.025145,-0.021415,-0.120477,0.004164
2023-05-05,0.014827,0.017199,0.003608,0.004547,-0.003190,0.018903,0.023444,0.037430,0.029358,0.037294,...,0.009355,0.037113,0.014005,0.006656,0.024828,0.008583,0.033208,0.009368,0.192173,0.043481


## Calculate daily returns of long worst 5 strategy
1. buy the assets with the worst 5 returns at close (at open the next day)
2. Sell at close the next day
Assumes no slippage between close[t] and open[t+1]

In [38]:
trades=np.zeros(df.shape[0])

In [39]:
for t in np.arange(1, df.shape[0]-2):
    worst5=dailyret.iloc[t].nsmallest(5).index
    trades[t+1]=df.iloc[t+1][worst5].sum()-df.iloc[t][worst5].sum()

In [40]:
# average daily return
trades[2:].mean()

-0.15417849999999494

In [41]:
# annualized Sharpe ratio, assuming 4% annual return on risk-free basis
trades_dailyret = pd.Series(trades[1:]).pct_change()

In [42]:
trades_dailyret_net = trades_dailyret - 0.04/252

In [43]:
sharpe = np.sqrt(252)*np.mean(trades_dailyret_net)/np.std(trades_dailyret_net)

In [44]:
sharpe

nan

In [45]:
trades_dailyret_net

0            NaN
1           -inf
2     -35.779056
3      -0.759083
4      -0.732918
         ...    
244    -1.522895
245     0.029243
246    -0.839444
247    -3.376224
248    -1.000159
Length: 249, dtype: float64

In [46]:
trades_dailyret_net.mean()

-inf

In [47]:
trades_dailyret_net.mean()/trades_dailyret_net.std()

nan

In [48]:
np.sqrt(250)*(trades_dailyret_net.mean() / trades_dailyret_net.std())

nan

## read SPX index returns over 250 days
### Get excess returns from "long worst 5" strategy by evaluating it directly against actual returns from the S&P 500 index

In [57]:
spx=pd.read_csv("^SPX.csv")

In [59]:
spx['Date']=spx['Date'].astype('datetime64')

In [60]:
spx.set_index('Date', inplace=True)

In [61]:
spx.sort_index(inplace=True)

In [62]:
riskfree_ret=spx['Adj Close'].pct_change().mean()

In [63]:
excessret = trades_dailyret-riskfree_ret

In [65]:
sharpe_ratio=np.sqrt(252)*excessret.iloc[2:].mean()/excessret.iloc[2:].std()

In [66]:
sharpe_ratio

-1.7209194545655262